In [1]:
import sys
sys.path.append(r"C:\\Users\\a-sochat\\development\\pysweep2.0")

import numpy as np
import pandas as pd

import pysweep
from pysweep import sweep, Measurement

import qcodes 

In [2]:
class NoisyResistor(qcodes.Instrument): 
    def __init__(self, name, resistance):
        super().__init__(name)
    
        self._resistance = resistance
        self._bias = 0
    
        self.add_parameter(
            name="bias", 
            set_cmd=self._set_bias,
            unit="V"
        )
        
        self.add_parameter(
            name="current", 
            get_cmd=self._get_current, 
            unit="A"
        )
        
        self.connect_message()
        
    def _set_bias(self, bias): 
        self._bias = bias
    
    def _get_current(self): 
        return self._bias / self._resistance + np.random.normal(0.0, 0.1)
    
    def connect_message(self, idn_param='IDN', begin_time=None): 
        
        con_msg = ('Connected to: noisy resistor')
        print(con_msg)
        return {}

r = NoisyResistor("R", 10)

Connected to: noisy resistor


In [3]:
station = qcodes.Station(r)

In [4]:
Measurement.use_storage("pandas")
Measurement.set_station(station)

In [5]:
setup = lambda s, n: {}
cleanup = lambda s, n: {}
run = lambda s, n, v: {"run": {"unit": "#", "value": v, "independent_parameter": True}}

In [6]:
run_n_times = lambda count: sweep(run, np.arange(count))

In [7]:
measurement = Measurement(
    setup, 
    cleanup, 
    run_n_times(100)(
        sweep(r.bias, np.linspace(0, 10, 5))(
            r.current
        )
    )
)

In [8]:
out = measurement.run()

In [10]:
df = out["R_current"]

In [11]:
df.groupby("R_bias [V]").mean()  # At each bias voltage, calculate the mean measured current. This averages over the 
# 100 repeats. 

,run [#],R_current [A]
R_bias [V],,
0.0,49.5,-0.006838
2.5,49.5,0.246173
5.0,49.5,0.501639
7.5,49.5,0.759037
10.0,49.5,0.987799


In [12]:
df

,R_bias [V],run [#],R_current [A]
0,0.0,0,-0.135807
1,2.5,0,0.281631
2,5.0,0,0.437883
3,7.5,0,0.755730
4,10.0,0,1.028413
5,0.0,1,0.088260
6,2.5,1,0.280027
7,5.0,1,0.633777
8,7.5,1,0.829426
9,10.0,1,0.936119
